<a href="https://colab.research.google.com/github/sonipranesh/Gen_AI_Projects/blob/main/chat_with_pdf_gen_ai_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai python-dotenv langchain PyPDF2 chromadb faiss-cpu langchain_google_genai -q

In [ ]:
!pip install langchain


In [ ]:
!pip install langchain-community

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain

from langchain.prompts import PromptTemplate

import google.generativeai as genai
from google.colab import files
import numpy as np
import json
import textwrap
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
from google.colab import userdata
from IPython.display import display , Markdown

In [ ]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA

In [ ]:
genai.configure(api_key=("Your API KEY")) #authorization

In [ ]:


uploaded = files.upload()
for _ in uploaded.keys():
  path='/content/'+ _

Saving ChatBot.pdf to ChatBot.pdf


In [ ]:
print(os.listdir('/content/'))

['.config', '349171_Ashwini_Saurabh_Resume.docx', 'ChatBot.pdf', 'sample_data']


In [ ]:
os.getcwd()

'/content'

In [ ]:
path = '/content/ChatBot.pdf'

In [ ]:
#path = '/content/ChatBot.pdf'
def get_pdf_text(path):
    text = ""
    pdf_reader = PdfReader(path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [ ]:
get_pdf_text(path)

"Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology giant Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak's Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the ﬁrst highly successful mass-produced microcomputers. Jobs saw the commercial potential of the Xerox Alto in 1979, which was mouse-driven and had a 

In [ ]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
#embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key)
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key="Your API KEY")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [ ]:
vectr=get_vector_store(get_text_chunks(get_pdf_text('/content/ChatBot.pdf')))

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7a97956399c0>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/grpc/_plugin_wrapping.py", line 105, in __call__
    self._metadata_plugin(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/transport/grpc.py", line 95, in __call__
    callback(self._get_authorization_headers(context), None)
  File "/usr/local/lib/python3.10/dist-packages/grpc/_plugin_wrapping.py", line 74, in __call__
    self._callback(metadata, cygrpc.StatusCode.ok, None)
  File "src/python/grpcio/grpc/_cython/_cygrpc/credentials.pyx.pxi", line 52, in grpc._cython.cygrpc._get_metadata.callback
  File "src/python/grpcio/grpc/_cython/_cygrpc/metadata.pyx.pxi", line 43, in grpc._cython.cygrpc._store_c_metadata
  File "src/python/grpcio/grpc/_cython/_cygrpc/grpc_string.pyx.pxi", line 40, in grpc._cython.cygrpc._encode
TypeError: Expected str, not 

GoogleGenerativeAIError: Error embedding content: 504 Deadline Exceeded

In [ ]:
import os

os.makedirs('faiss_index', exist_ok=True)


In [ ]:
!ls faiss_index

In [ ]:
import shutil

shutil.rmtree('faiss_index')

In [ ]:
def get_qa_chain():

    prompt_template = """
    Given a pre-processed PDF document embedding context, answer the user's question Question in a way that simulates reading and understanding a storybook. Answers should be limited to 20 words per line.
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key="Your API KEY")

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:

user_query = input("State your question: ")
print("You asked:", user_query)

query=get_pdf_text(path)
text_chunks=get_text_chunks(query)
get_vector_store(text_chunks)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="Your API KEY")
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_query)

print(docs)


chain = get_qa_chain()

response = chain(
  {"input_documents": docs, "question": user_query},
  return_only_outputs=True
)

#print(response)
#print("Reply: ", response["output_text"])

response["output_text"]

State your question: who is Steve jobs


ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7a97954df250>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/grpc/_plugin_wrapping.py", line 105, in __call__
    self._metadata_plugin(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/transport/grpc.py", line 95, in __call__
    callback(self._get_authorization_headers(context), None)
  File "/usr/local/lib/python3.10/dist-packages/grpc/_plugin_wrapping.py", line 74, in __call__
    self._callback(metadata, cygrpc.StatusCode.ok, None)
  File "src/python/grpcio/grpc/_cython/_cygrpc/credentials.pyx.pxi", line 52, in grpc._cython.cygrpc._get_metadata.callback
  File "src/python/grpcio/grpc/_cython/_cygrpc/metadata.pyx.pxi", line 43, in grpc._cython.cygrpc._store_c_metadata
  File "src/python/grpcio/grpc/_cython/_cygrpc/grpc_string.pyx.pxi", line 40, in grpc._cython.cygrpc._encode
TypeError: Expected str, not 

You asked: who is Steve jobs


GoogleGenerativeAIError: Error embedding content: 504 Deadline Exceeded

In [ ]:
res=(response["output_text"])
print (res)

In [ ]:
type(docs)
new_db.similarity_search('alice')

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'Gapi'

In [ ]:
from googleapiclient.discovery import build

In [ ]:
retriever = new_db.as_retriever()
docs = retriever.invoke({"question": user_query})

In [ ]:
user_query="find the author"
docs = new_db.similarity_search(user_query)


chain = get_qa_chain()

response = chain(
  {"input_documents": docs, "question": user_query},
  return_only_outputs=True
)

print(response)
print("Reply: ", response["output_text"])


In [ ]:
def ask_and_answer(faiss_index, google_api_key):
  """Prompts the user for a question, retrieves relevant text chunks, and generates an answer.

  Args:
      faiss_index: The FAISS index containing pre-computed similarity searches.
      google_api_key: Your Google Generative AI API key.

  Returns:
      None
  """

  # Prompt for user question
  user_query = input("State your question: ")

  # Retrieve relevant text chunks using the FAISS index
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="Your API KEY")
  docs = faiss_index.similarity_search(user_query)  # Assuming similarity_search returns relevant documents

  # Prepare the prompt template (adjust as needed)
  prompt_template = PromptTemplate("""
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
  """)

  # You might already have a question answering chain defined elsewhere
  # Replace this section with your existing logic for invoking the chain
  # ... (your question answering chain logic using prompt_template, docs, and user_query)

  # Print the answer (assuming the chain provides the answer in a specific variable)
  # print(response)
  print("Reply: ", answer_variable)  # Replace with the actual variable holding the answer

if __name__ == "__main__":
  # Assuming faiss_index is loaded elsewhere (from previous code execution)
  ask_and_answer(faiss_index, google_api_key)

In [ ]:
print(get_vector_store.response())

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def user_input():

    user_query = input("state your question: ")

    new_db=FAISS.load_local("faiss_index", embeddings)
    docs=new_db.similarity_search(user_query)

    chain= get_qa_chain()

    response= chain(
        {"input_docuents":docs, "question": user_query}
        , return_only_outputs=True)

    print(response)

    print("Reply: ", response["output_text"])


In [ ]:
text=input("your question please: ")
print (text)

In [ ]:
def find_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader=PdfReader(pdf)
        for page in pdf_reader.pages:
            text+=page.extract_text()
    return text

uploaded = files.upload()
for _ in uploaded.keys():
  path='/content/'+ _


In [ ]:
import PyPDF2

def to_markdown(text):
  text=text.replace('.', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def find_pdf_text(pdf_file):
    """Extracts text from a single PDF file and returns it as a string."""

    try:
        with open(pdf_file, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return to_markdown(text) # Remove leading/trailing whitespace

    except Exception as e:
        print(f"An error occurred while processing the PDF: {e}")
        return "Error: Unable to extract text from PDF."


# Example usage (assuming you've uploaded the PDF to Colab):
uploaded = files.upload()
pdf_path = '/content/' + list(uploaded.keys())[0]  # Get the uploaded file path
extracted_text = find_pdf_text(pdf_path)

if extracted_text != "Error: Unable to extract text from PDF.":
  print("Extracted Text:")
  print(extracted_text)
else:
  print(extracted_text)  # Display the error message if extraction failed


In [ ]:
!pip install PyMuPDF
!pip install pillow
!wget https://pubs.usgs.gov/circ/1441/circ1441.pdf


In [ ]:
import os
import pymupdf
import PIL
import google.generativeai as genai


In [ ]:
os.getcwd()

# Define the folder name for saving images
pics_folder = "pics"

# Create the pics folder if it doesn't exist
if not os.path.exists(pics_folder):
  os.makedirs(pics_folder)  # Create the folder

doc = pymupdf.open("circ1441.pdf")  # Open a document

for page_index in range(len(doc)):
  page = doc[page_index]  # Get the page
  image_list = page.get_images()

  # Print the number of images found on the page
  if image_list:
    print(f"Found {len(image_list)} images on page {page_index}")
  else:
    print("No images found on page", page_index)

  for image_index, img in enumerate(image_list, start=1):
    xref = img[0]  # Get the XREF of the image
    pix = pymupdf.Pixmap(doc, xref)  # Create a Pixmap

    if pix.n - pix.alpha > 3:  # CMYK: convert to RGB first
      pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

    # Save the image with a filename format and path to pics folder
    filename = f"{pics_folder}/page_{page_index}-image_{image_index}.png"
    pix.save(filename)  # Save the image as png
    pix = None

In [ ]:
image1 = PIL.Image.open('/content/pics/page_0-image_1.png')
image2 = PIL.Image.open('/content/pics/page_0-image_10.png')
image3 = PIL.Image.open('/content/pics/page_0-image_11.png')


In [ ]:

model = genai.GenerativeModel("gemini-pro-vision",generation_config={"temperature":1})

response = model.generate_content(
[f"""Your task is to provide a summary of the given pictures. use details to describle the provided image and also use context wherever applicable or necessary


""",
image1,image2,image3])
print(response.text)